<a href="https://www.kaggle.com/code/safaafathii/loan-eligibility-prediction?scriptVersionId=113308943" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [65]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/loan-prediction-practice-av-competition/sample_submission.csv
/kaggle/input/loan-prediction-practice-av-competition/test.csv.csv
/kaggle/input/loan-prediction-practice-av-competition/train_csv.csv


# Importing Libraries

In [66]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import xgboost as xgb

# Reading Data

In [67]:
df_train = pd.read_csv("../input/loan-prediction-practice-av-competition/train_csv.csv")
df_train.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [68]:
df_test = pd.read_csv("../input/loan-prediction-practice-av-competition/test.csv.csv")
df_test.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,LP001015,Male,Yes,0,Graduate,No,5720,0,110.0,360.0,1.0,Urban
1,LP001022,Male,Yes,1,Graduate,No,3076,1500,126.0,360.0,1.0,Urban
2,LP001031,Male,Yes,2,Graduate,No,5000,1800,208.0,360.0,1.0,Urban
3,LP001035,Male,Yes,2,Graduate,No,2340,2546,100.0,360.0,NaN,Urban
4,LP001051,Male,No,0,Not Graduate,No,3276,0,78.0,360.0,1.0,Urban


In [69]:
df_train.shape

(614, 13)

In [70]:
df_test.shape

(367, 12)

# Exploratory Data Analysis

In [71]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            614 non-null    object 
 1   Gender             601 non-null    object 
 2   Married            611 non-null    object 
 3   Dependents         599 non-null    object 
 4   Education          614 non-null    object 
 5   Self_Employed      582 non-null    object 
 6   ApplicantIncome    614 non-null    int64  
 7   CoapplicantIncome  614 non-null    float64
 8   LoanAmount         592 non-null    float64
 9   Loan_Amount_Term   600 non-null    float64
 10  Credit_History     564 non-null    float64
 11  Property_Area      614 non-null    object 
 12  Loan_Status        614 non-null    object 
dtypes: float64(4), int64(1), object(8)
memory usage: 62.5+ KB


In [72]:
df_train.describe()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History
count,614.000000,614.000000,592.000000,600.00000,564.000000
mean,5403.459283,1621.245798,146.412162,342.00000,0.842199
std,6109.041673,2926.248369,85.587325,65.12041,0.364878
min,150.000000,0.000000,9.000000,12.00000,0.000000
25%,2877.500000,0.000000,100.000000,360.00000,1.000000
50%,3812.500000,1188.500000,128.000000,360.00000,1.000000
75%,5795.000000,2297.250000,168.000000,360.00000,1.000000
max,81000.000000,41667.000000,700.000000,480.00000,1.000000


In [73]:
df_train.dtypes

Loan_ID               object
Gender                object
Married               object
Dependents            object
Education             object
Self_Employed         object
ApplicantIncome        int64
CoapplicantIncome    float64
LoanAmount           float64
Loan_Amount_Term     float64
Credit_History       float64
Property_Area         object
Loan_Status           object
dtype: object

In [74]:
df_train.isnull().sum()

Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

## Handle Null Values

In [75]:
df_train.select_dtypes(include=["object"]).columns

Index(['Loan_ID', 'Gender', 'Married', 'Dependents', 'Education',
       'Self_Employed', 'Property_Area', 'Loan_Status'],
      dtype='object')

In [76]:
for col in df_train.select_dtypes(include=["object"]):
    df_train[col] = df_train[col].fillna(df_train[col].mode()[0])
    print (col)

Loan_ID
Gender
Married
Dependents
Education
Self_Employed
Property_Area
Loan_Status


In [77]:
df_train.isnull().sum()

Loan_ID               0
Gender                0
Married               0
Dependents            0
Education             0
Self_Employed         0
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [78]:
df_train["LoanAmount"].value_counts()

120.0    20
110.0    17
100.0    15
160.0    12
187.0    12
         ..
240.0     1
214.0     1
59.0      1
166.0     1
253.0     1
Name: LoanAmount, Length: 203, dtype: int64

In [79]:
df_train["LoanAmount"] = df_train["LoanAmount"].fillna(df_train["LoanAmount"].median())

In [80]:
df_train["Loan_Amount_Term"].value_counts()

360.0    512
180.0     44
480.0     15
300.0     13
240.0      4
84.0       4
120.0      3
60.0       2
36.0       2
12.0       1
Name: Loan_Amount_Term, dtype: int64

In [81]:
df_train["Loan_Amount_Term"] = df_train["Loan_Amount_Term"].fillna(df_train["Loan_Amount_Term"].mode()[0])

In [82]:
df_train["Credit_History"].value_counts()

1.0    475
0.0     89
Name: Credit_History, dtype: int64

In [83]:
df_train["Credit_History"] = df_train["Credit_History"].fillna(df_train["Credit_History"].mode()[0])

In [84]:
df_train.isnull().sum()

Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

### -------------------------------------------------------------------------------

## One Hot Encoding

In [85]:
df_train.nunique()

Loan_ID              614
Gender                 2
Married                2
Dependents             4
Education              2
Self_Employed          2
ApplicantIncome      505
CoapplicantIncome    287
LoanAmount           203
Loan_Amount_Term      10
Credit_History         2
Property_Area          3
Loan_Status            2
dtype: int64

In [86]:
df_train = pd.get_dummies(df_train,columns=["Dependents","Property_Area"])

In [87]:
df_train

,Loan_ID,Gender,Married,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Loan_Status,Dependents_0,Dependents_1,Dependents_2,Dependents_3+,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban
0,LP001002,Male,No,Graduate,No,5849,0.0,128.0,360.0,1.0,Y,1,0,0,0,0,0,1
1,LP001003,Male,Yes,Graduate,No,4583,1508.0,128.0,360.0,1.0,N,0,1,0,0,1,0,0
2,LP001005,Male,Yes,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Y,1,0,0,0,0,0,1
3,LP001006,Male,Yes,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Y,1,0,0,0,0,0,1
4,LP001008,Male,No,Graduate,No,6000,0.0,141.0,360.0,1.0,Y,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,LP002978,Female,No,Graduate,No,2900,0.0,71.0,360.0,1.0,Y,1,0,0,0,1,0,0
610,LP002979,Male,Yes,Graduate,No,4106,0.0,40.0,180.0,1.0,Y,0,0,0,1,1,0,0
611,LP002983,Male,Yes,Graduate,No,8072,240.0,253.0,360.0,1.0,Y,0,1,0,0,0,0,1
612,LP002984,Male,Yes,Graduate,No,7583,0.0,187.0,360.0,1.0,Y,0,0,1,0,0,0,1


## Label Encoding

In [88]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df_train["Gender"] = le.fit_transform(df_train["Gender"])
le = LabelEncoder()
df_train["Married"] = le.fit_transform(df_train["Married"])
le = LabelEncoder()
df_train["Education"] = le.fit_transform(df_train["Education"])
le = LabelEncoder()
df_train["Self_Employed"] = le.fit_transform(df_train["Self_Employed"])
le = LabelEncoder()
df_train["Loan_Status"] = le.fit_transform(df_train["Loan_Status"])

# Splitting Data

In [89]:
df_train = df_train.drop("Loan_ID",axis=1)

In [90]:
X = df_train.drop("Loan_Status",axis=1)
y = df_train["Loan_Status"]

In [91]:
# from sklearn.decomposition import PCA
# pca = PCA(n_components=2)
# X = pca.fit_transform(X)

In [92]:
y = y.replace("Y",1)
y = y.replace("N",0)

In [93]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33, random_state=42)

# XGBoost

In [94]:
model = xgb.XGBClassifier()
model.fit(X_train,y_train)
y_train_Pred = model.predict(X_train)
predicted_y = model.predict(X_val)

In [95]:
y.value_counts()

1    422
0    192
Name: Loan_Status, dtype: int64

In [96]:
from sklearn.metrics import f1_score
#y_pred_class = y_pred_pos > threshold
#print("Training F1 Score: " + str(f1_score(y_train, y_train_Pred)))
print("Validation F1 Score: " + str(f1_score(y_val, predicted_y)))

Validation F1 Score: 0.8472222222222222


# Logistic Regression

In [121]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=42,C = 2).fit(X_train, y_train)
Y_Log_Pred = clf.predict(X_val)

In [122]:
print("Validation F1 Score: " + str(f1_score(y_val, Y_Log_Pred)))

Validation F1 Score: 0.8628762541806019


# Random Forest Classifier

In [123]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X_train, y_train)
Y_Rand_Pred = clf.predict(X_val)

In [124]:
print("Validation F1 Score: " + str(f1_score(y_val, Y_Rand_Pred)))

Validation F1 Score: 0.8543046357615894
